In [21]:
import jieba

# 文件路径
file_path = 'toutiao_cat_data.txt'

documents = []
all_texts = []

# 打开文件并按行处理
with open(file_path, encoding='utf-8') as file:
    for line in file:
        # 使用jieba进行分词
        # 提取第三个元素，通过感叹号分割，然后去掉空格并进行分词
        words = jieba.cut(line.strip().split('_!_')[3].strip())
        document = list(words)

        documents.append(document)
        all_texts += document

# 文档数和总单词数
doc_size = len(documents)
text_size = len(all_texts)

print(doc_size)
print(text_size)

382688
5090680


In [22]:
categories = {
    '100': '民生故事',
    '101': '文化',
    '102': '娱乐',
    '103': '体育',
    '104': '财经',
    '106': '房产',
    '107': '汽车',
    '108': '教育',
    '109': '科技',
    '110': '军事',
    '112': '旅游',
    '113': '国际',
    '114': '证券股票',
    '115': '农业三农',
    '116': '电竞游戏'
}

from gensim import corpora

# 文書データから辞書を作成
dictionary = corpora.Dictionary(documents)

# 全テキストをBoWにすると，各単語の出現数を数えたことになる
bow = dictionary.doc2bow(all_texts)

# BoWに含まれる単語の種類数（語彙サイズ）は
vocab_size = len(bow)
print(vocab_size)

# 先頭の10単語を眺めてみる（単語IDと頻度）
# 自然言語処理の世界では，単語をIDすなわち番号で表す習慣（というより必要性）が昔からあった
bow[0:10]

169696


[(0, 299),
 (1, 33),
 (2, 35956),
 (3, 2337),
 (4, 276),
 (5, 1272),
 (6, 10544),
 (7, 12),
 (8, 150717),
 (9, 57)]

In [23]:
# 単語がID化されているので，単語そのものに戻しておく
all_texts_wf = [ (dictionary.get(w[0]), w[1]) for w in dictionary.doc2bow(all_texts) ]

# すると先ほどの単語10個は
all_texts_wf[0:10]

[('之旅', 299),
 ('京城', 33),
 ('你', 35956),
 ('值得', 2337),
 ('博物馆', 276),
 ('文化', 1272),
 ('最', 10544),
 ('来场', 12),
 ('的', 150717),
 ('发酵', 57)]

In [24]:
# 頻度を降順に並べ替えて眺めてみる
all_texts_wf_sorted = sorted(all_texts_wf, key=lambda wf: wf[1], reverse=True)
for i in range(50):
    print(all_texts_wf_sorted[i])

('，', 267580)
('？', 188603)
('的', 150717)
('！', 70185)
('：', 60396)
(' ', 53018)
('了', 50236)
('是', 40195)
('你', 35956)
('“', 34985)
('”', 34898)
('吗', 30161)
('有', 30114)
('在', 26648)
('如何', 22601)
('什么', 21552)
('怎么', 20168)
('为什么', 19554)
('和', 18842)
('都', 17407)
('中国', 17050)
('》', 16557)
('《', 16546)
('被', 16408)
('对', 16034)
('不', 15898)
('人', 15423)
('、', 13982)
('会', 13724)
('我', 13421)
('看', 12626)
('年', 12450)
('这', 11749)
('美国', 10913)
('能', 10698)
('上', 10585)
('最', 10544)
('上联', 10138)
('好', 10042)
('下联', 9987)
('要', 9681)
('还', 9643)
('后', 9468)
('大', 9278)
('就', 9238)
('5', 8818)
('中', 8803)
('哪些', 8776)
('将', 8678)
('与', 8637)


In [25]:
# 今度は文書ごとにBoWにする
doc_bow = [dictionary.doc2bow(d) for d in documents]

# 保存もできる
corpora.MmCorpus.serialize('diet/diet.mm', doc_bow)
# ある1つの文書の単語頻度を今度は昇順に並べ替えて，眺めてみる
# たとえば最初（0番目）の文書
doc_id = 0

wv = [(dictionary.get(w[0]), w[1]) for w in sorted(doc_bow[doc_id], key=lambda wf: wf[1])]

# この文書の単語の種類数
# 文書に出現した単語しか含まれない（したがって全体の語彙サイズより小さい）ことに注意

# スペースの節約のため，つなげてから表示
wv_seq = ""
for i in range(len(wv)):
    # 単語の後にコロン（：）で区切って頻度を付けて表示
    wv_seq += " " + wv[i][0] + ":" + str(wv[i][1])
print(wv_seq)

 之旅:1 京城:1 你:1 值得:1 博物馆:1 文化:1 最:1 来场:1 的:1


In [26]:
# データフレームに変換する場合
import numpy as np
from pandas import DataFrame 

# 空（0）の行列を用意して，各文書で単語が出現していた場合だけその頻度を代入
word_vect = np.zeros(doc_size * vocab_size, dtype=np.int32).reshape(doc_size, vocab_size)
for i in range(doc_size):
    for w, f in doc_bow[i]:
        word_vect[i][w] = f

# データフレームに変換して列名に単語を設定
word_df = DataFrame(word_vect)
word_df.columns=[ dictionary.get(i) for i in range(vocab_size)]
word_df

MemoryError: Unable to allocate 242. GiB for an array with shape (64940622848,) and data type int32

In [ ]:
# この単語頻度ベクトルがそれぞれの文書を表しているので，これにより分類などを行うことが（いちおう）できる

# 読点「、」と句点「。」がどの文書でも著しく頻度が大きいので，あらかじめ抜いておく
del word_df['、']
del word_df['。']

word_vect = np.array(word_df)


# コサイン類似度を定義して
def cos_similarity(v1, v2):
    return np.dot(v1, v2) / np.linalg.norm(v1) / np.linalg.norm(v2)


# 文書間の類似度を計算
wv_sims = [[cos_similarity(i, j) for j in word_vect] for i in word_vect]
# たとえば最初の文書と各文書との類似度を見ると，どれもあまり変わらない…
# （これは，句読点は外したが，どの文書にも出てくる機能語などはそのままにしたため）
wv_sims[0]

In [ ]:
#TF-IDF
from gensim import models

# 読み込んだ文書データからIDFを計算
tfidf = models.TfidfModel(doc_bow)

# これをBoWのデータ（つまりTF）に適用するとTF-IDFが計算できる
# 同じ文書集合で計算してみる（もちろん別の文書でもできる）
tfidf_vect = tfidf[doc_bow]

# たとえば最初の文書について，先ほどのBoWとTF-IDFとで単語の種類数を見てみると
print(len(doc_bow[0]))
print(len(tfidf_vect[0]))

# 大きさが異なる（TF-IDFの種類の方が少ない）ことに注意
# これは，TF-IDFの計算時に，全文書に出てくる単語が（TF-IDFが0になるので）省かれるため

In [ ]:
# ある1つの文書の単語をいくつか抜き出してTF-IDF値を見てみる
# （値が正規化されていることに注意）
# たとえば最初（0番目）の文書
doc_id = 0

for i in range(0, 3000, 50):
    print("%-8s\t%.6f" % (dictionary.get(tfidf_vect[doc_id][i][0]), tfidf_vect[doc_id][i][1]))

In [ ]:
# TF-IDFによる文書のベクトルが定義できたので，類似度を計算してみる
from gensim import similarities

index = similarities.MatrixSimilarity(tfidf_vect)

# 同じ文書集合に対する類似度（他の文書を用意して計算することも，もちろんできる）
sims = index[tfidf_vect]
# 最初の文書に関する類似度
print(sims[0])

# 単純な単語頻度ベクトルよりはメリハリがついた

In [ ]:
for i in range(5):
    print(f"Document {i + 1}: {documents[i]}")

In [ ]:
# 最初の辞書から，文書頻度が一定以下・一定以上のものを除去
documents = documents[:10000]
dictionary.filter_extremes()

# 新しい辞書のサイズ
print(len(list(dictionary.values())))

In [30]:
# LDAで学習された各トピックを特徴付ける単語（それぞれ重み，すなわち確率を持っている）
# クラスタリングにおけるクラスタの番号とは全く関係がないので注意
import re
topic_words = []
documents = documents[:10000]
documents=re.sub(r'[^\w\s]', '', documents)
topic_num=10
word_vect2 = [ dictionary.doc2bow(d) for d in documents ]
lda = models.LdaModel(word_vect2, num_topics=topic_num, passes=10)

for i in range(topic_num):
    topic_words.append(" ".join([ "%s@%.3f" % (dictionary.get(t[0]), t[1]) for t in lda.get_topic_terms(i) ] ))
    print("%d: %s" % (i, topic_words[-1]))


TypeError: expected string or bytes-like object

In [35]:
import re
from gensim import corpora, models

documents = [str(sentence) for sentence in documents]

# 将文档列表转换为字符串
documents_text = " ".join(documents)

# 使用正则表达式移除标点符号
documents_text = re.sub(r'[^\w\s]', '', documents_text)

# 将处理后的文本拆分为单词
preprocessed_documents = documents_text.split()

# 创建字典和文档向量
dictionary = corpora.Dictionary([preprocessed_documents])
word_vect2 = [dictionary.doc2bow(preprocessed_documents)]

# 使用LDA模型
topic_num = 10
lda = models.LdaModel(word_vect2, num_topics=topic_num, passes=10)

# 打印主题词
topic_words = []
for i in range(topic_num):
    topic_words.append(" ".join(["%s@%.3f" % (dictionary.get(t[0]), t[1]) for t in lda.get_topic_terms(i)]))
    print("%d: %s" % (i, topic_words[-1]))

0: 的@0.000 是@0.000 了@0.000 有@0.000 你@0.000 怎么@0.000 吗@0.000 在@0.000 什么@0.000 如何@0.000
1: 的@0.001 是@0.000 吗@0.000 了@0.000 你@0.000 怎么@0.000 在@0.000 有@0.000 什么@0.000 如何@0.000
2: 的@0.002 了@0.001 是@0.001 在@0.001 你@0.001 有@0.001 什么@0.001 吗@0.000 如何@0.000 怎么@0.000
3: 的@0.036 了@0.011 你@0.010 是@0.009 吗@0.008 有@0.008 在@0.007 怎么@0.007 什么@0.006 如何@0.006
4: 的@0.002 了@0.001 你@0.001 如何@0.001 吗@0.001 是@0.001 有@0.001 怎么@0.000 什么@0.000 在@0.000
5: 的@0.026 了@0.008 有@0.007 是@0.006 吗@0.006 你@0.006 怎么@0.005 为什么@0.005 在@0.004 什么@0.004
6: 的@0.000 了@0.000 你@0.000 吗@0.000 有@0.000 是@0.000 在@0.000 怎么@0.000 如何@0.000 为什么@0.000
7: 的@0.001 吗@0.000 你@0.000 是@0.000 了@0.000 怎么@0.000 在@0.000 有@0.000 为什么@0.000 如何@0.000
8: 的@0.001 你@0.000 有@0.000 吗@0.000 了@0.000 是@0.000 怎么@0.000 如何@0.000 和@0.000 什么@0.000
9: 的@0.001 了@0.000 吗@0.000 有@0.000 你@0.000 怎么@0.000 是@0.000 在@0.000 什么@0.000 和@0.000


In [38]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\64403\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [43]:
import re
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
file_path = 'toutiao_cat_data.txt'

documents = []
all_texts = []

# 打开文件并按行处理
with open(file_path, encoding='utf-8') as file:
    for line in file:
        # 使用jieba进行分词
        # 提取第三个元素，通过感叹号分割，然后去掉空格并进行分词
        words = jieba.cut(line.strip().split('_!_')[3].strip())
        document = list(words)

        documents.append(document)

# 将每个句子转换为字符串
documents = [str(sentence) for sentence in documents]

# 将文档列表转换为字符串
documents_text = " ".join(documents)

# 使用正则表达式移除标点符号
documents_text = re.sub(r'[^\w\s]', '', documents_text)

# 将处理后的文本拆分为单词
preprocessed_documents = documents_text.split()

# 合并停用词列表
stop_words = ['的', '是', '在', '了', '和', '与', '及', '他', '她', '它', '我们', '你们', '自己', '这', '那', '什么', '为什么', '如何', '怎么', '哪些', '上联', '哪个', '应该', '多少', '一个', '下联', '还是', '如果', '为何', '怎样', '到底', '不能', '知道', '就是', '影响', '怎么样', '哪里', '这个', '的', '是','你','有','吗','人','都','不']

# 移除停用词
preprocessed_documents = [word for word in preprocessed_documents if word.lower() not in stop_words]


# 创建字典和文档向量
dictionary = corpora.Dictionary([preprocessed_documents])
word_vect2 = [dictionary.doc2bow(preprocessed_documents)]

# 使用LDA模型
topic_num = 10
lda = models.LdaModel(word_vect2, num_topics=topic_num, passes=10)

# 打印主题词
topic_words = []
for i in range(topic_num):
    topic_words.append(" ".join(["%s@%.3f" % (dictionary.get(t[0]), t[1]) for t in lda.get_topic_terms(i)]))
    print("%d: %s" % (i, topic_words[-1]))


0: 中国@0.001 对@0.001 会@0.001 好@0.000 我@0.000 美国@0.000 能@0.000 大@0.000 上@0.000 还@0.000
1: 对@0.001 我@0.001 中国@0.001 会@0.001 好@0.000 被@0.000 看@0.000 就@0.000 能@0.000 最@0.000
2: 被@0.002 年@0.002 中国@0.002 对@0.002 看@0.001 我@0.001 会@0.001 月@0.001 最@0.001 还@0.001
3: 被@0.001 中国@0.001 对@0.001 看@0.001 年@0.001 美国@0.001 要@0.001 会@0.001 能@0.001 上@0.001
4: 对@0.001 被@0.001 中国@0.001 会@0.001 我@0.001 上@0.001 美国@0.001 最@0.001 还@0.001 5@0.001
5: 被@0.001 对@0.001 中国@0.001 我@0.001 会@0.001 上@0.001 好@0.001 手机@0.001 看@0.001 后@0.001
6: 会@0.001 年@0.001 中国@0.001 能@0.000 5@0.000 对@0.000 最@0.000 被@0.000 看@0.000 美国@0.000
7: 被@0.001 会@0.001 中国@0.001 对@0.001 我@0.001 年@0.001 看@0.001 美国@0.001 上@0.001 后@0.001
8: 中国@0.002 被@0.002 会@0.002 对@0.002 上@0.001 我@0.001 年@0.001 手机@0.001 大@0.001 看@0.001
9: 中国@0.005 被@0.005 对@0.005 会@0.004 我@0.004 看@0.004 年@0.004 美国@0.003 能@0.003 上@0.003
